In [1]:
import numpy as np
from numpy.linalg import norm
from numpy.random import normal, beta, choice, permutation
from numpy.typing import NDArray
from scipy.special import gamma, digamma
from typing import Callable

# Стохастический градиентный спуск RMSProp

In [2]:
from RMSProp import SGD_RMSProp

### Тест SGD RMSPror

In [3]:
def L(w, X, y):
    return norm(X.dot(w) - y) ** 2 / y.size


def L_grad(w, X, y):
    return 2 * X.T.dot(X.dot(w) - y) / y.size


np.random.seed(42)
nrow, ncol = 500, 4
X = normal(0, 1, ncol * nrow).reshape(nrow, ncol)
X_ones = np.hstack([X, np.ones((nrow, 1))])
true_w = np.array([2, -3, 1, 0.5, 4])
y = X_ones.dot(true_w) + normal(0, 1, nrow)
w_start = normal(0, 1, ncol + 1)

sgd_rmsprop_res = SGD_RMSProp(
    start=w_start,
    X=X_ones,
    y=y,
    L_grad=L_grad,
    batch_size=100,
    decay_rate=0.9
)

In [4]:
print(f'Iterations: {sgd_rmsprop_res["iterations"]}')
print(f'w_e: {sgd_rmsprop_res["point"]}')
print(f'||w_e-w_t||^2 = {norm(sgd_rmsprop_res["point"] - true_w) ** 2}')

Iterations: 1000
w_e: [ 2.0051351  -3.06448662  0.92679217  0.53816992  3.99523889]
||w_e-w_t||^2 = 0.01102389173910001


## Функция правдоподобия для бета-регрессии
[Статья про бета-регрессию](https://www.ime.usp.br/~sferrari/beta.pdf)

Пусть $\xi\sim \Beta(\alpha, \beta)$.

**Плотность**: 
$$
    f_\xi(x)=\frac{x^{\alpha-1}(1-x)^{\beta-1}}{\Beta(\alpha, \beta)}, \qquad x \in (0, 1).
$$
Для построения бета-регрессии удобнее работать в параметризации через среднее и "точность":
$$
    \mu = \frac{\alpha}{\alpha + \beta}, \qquad \varphi = \alpha + \beta,\\
    \mu \in (0, 1), \qquad \varphi > 0.
$$
Тогда старые параметры выражаются следующим образом:
$$
    \alpha = \mu\varphi, \qquad \beta = (1-\mu)\varphi.
$$ 

Среднее и дисперсия хорошо выражаются через новые параметры:
$$
    \mathrm{E}(\xi) = \mu, \qquad \sigma^2 = \frac{\mu(1-\mu)}{1 + \varphi}
$$

**Плотность в новой параметризации**:
$$
    f_\xi(x) = \frac{\Gamma(\varphi)}{\Gamma(\mu\varphi)\Gamma((1-\mu)\varphi)}x^{\mu\varphi-1}(1-x)^{(1-\mu)\varphi-1}, \qquad x \in (0, 1).
$$

Пусть $\mathbf{X}\in \mathbb{R}^{n\times p}$ - выборка регрессоров, $Y \in \mathbb{R}^{n}$ - выборка откликов.
Предполагается, что $y_i \sim \Beta(\mu_i, \varphi)$, где параметр $\varphi$ неизвестен, а
$\mu_i$ выражается через регрессоры:
$$
    g(\mu_i) = \bold{x}^{\mathrm{T}}_i \boldsymbol{\beta}.
$$
$g(t)$ - произвольная линк-функция, например логит:
$$
    g(\mu_i) = \log\left( \frac{\mu_i}{1-\mu_i} \right) = \bold{x}^{\mathrm{T}}_i \boldsymbol{\beta} \implies 
    \mu_i = \frac{e^{\bold{x}^{\mathrm{T}}_i \boldsymbol{\beta}}}{1 + e^{\bold{x}^{\mathrm{T}}_i \boldsymbol{\beta}}}.
$$

**Логарифм функции правдоподобия**:
$$
    L(\mathbf{X}, \beta, \varphi; Y) = \sum_{i=1}^{n}l(\mu_i(\bold{x}_i, \beta), \varphi; y_i)\\

    l(\mu_i(\bold{x}_i, \beta), \varphi; y_i) = \log\Gamma(\varphi) - \log\Gamma(\mu_i\varphi) - \log\Gamma((1-\mu_i)\varphi) + 
    (\mu_i\varphi - 1) \log y_i + ((1 - \mu_i)\varphi - 1)\log (1 - y_i).
$$

## Градиент логарифма функции правдободобия
Пусть $\mathbf{X}$ и $Y$ фиксированы, обозначим 
$$y_i^* = \log(y_i / (1 - y_i)), \qquad \mu_i^*=\psi(\mu_i\varphi) - \psi((1 - \mu_i)\varphi), \qquad
\mathbf{T} = \mathrm{diag}\left( 1 / g'(\mu_1), \ldots, 1 / g'(\mu_n) \right),
$$
$$
  Y^* = (y_1^*, \ldots, y_n^*)^{\mathrm{T}}, \qquad \boldsymbol{\mu}^* = (\mu_1^*, \ldots, \mu_n^*)^{\mathrm{T}},
$$
где $\psi(z) = (\log\Gamma(z))'$ - дигамма-функция (есть в модуле `scipy.special`),
тогда градиент логарифма функции правдоподобия равен 
$\nabla L(\beta, \varphi) = \left( L_{\beta}^{\mathrm{T}}(\beta, \varphi), L_{\varphi}(\beta, \varphi) \right)^{\mathrm{T}}$, где
$$
    L_{\beta}(\beta, \varphi) = \varphi \mathbf{X}^{\mathrm{T}}\mathbf{T}(Y^* - \boldsymbol{\mu^*}),
$$
$$
  L_{\varphi}(\beta, \varphi) = \sum_{i=1}^n 
  \big(  
    \mu_i(y_i^* - \mu_i^*) + \log(1 - y_i) - \psi((1 - \mu_i)\varphi) + \psi(\varphi)
  \big).
$$

In [5]:
from BetaRegression import *

### Тест SGD RMSProp для бета регрессии

In [6]:
%%time

np.random.seed(42)
nrow, ncol = 500, 4
X = normal(0, 1, ncol * nrow).reshape(nrow, ncol)
X_ones = np.hstack([X, np.ones((nrow, 1))])
true_beta = np.array([0.1, 0.3, 0.01, 0.5, 0.4])
true_phi = 3
true_mu = logit_inverse(X_ones, true_beta)
y = np.array([beta(true_phi * mu, (1 - mu) * true_phi) for mu in true_mu])
w_start = np.append(normal(0, 1, ncol + 1), 2)

beta_res = SGD_RMSProp(
    start=w_start,
    X=X_ones,
    y=y,
    L_grad=beta_illh_grad,
    L=beta_inv_log_likelihood,
    batch_size=50,
    learning_rate=0.01,
    decay_rate=0.1,
    max_iter=10000,
    tol=1e-4,
    link_inverse=logit_inverse,
    link_deriv=logit_deriv,
)

CPU times: user 58.1 ms, sys: 1.09 ms, total: 59.2 ms
Wall time: 58.3 ms


In [7]:
print(f'Iterations: {beta_res["iterations"]}')
print(f'w_e = {beta_res["point"]}')
print(f'||w_e-w_t||^2 = {norm(beta_res["point"] - np.append(true_beta, true_phi)) ** 2}')

Iterations: 910
w_e = [0.10627906 0.34577996 0.02418946 0.47347655 0.43097665 3.03098291]
||w_e-w_t||^2 = 0.004959559800569143
